In [120]:
# Importing libraries
import pandas as pd 
import cbpro
import json
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import requests
import dateutil.parser
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime, date
import altair as alt
import json
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### Coinbase Pro API

In [121]:
# coinbase pro api keys
## hidden

In [122]:
# Public account information
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass)

public_client = cbpro.PublicClient()

### Kraken API

In [4]:
# Kraken api keys
## hidden

# Coinbase Pro 

### Getting investment data from Coinbase Pro account

In [5]:
# Retrieving crypto asset accounts
acc_raw = client.get_accounts() # getting all accounts 
accounts = []
for acc in acc_raw:
    balance = float(acc.get("balance"))
    if balance != 0:
        currency = acc.get("currency") 
        acc_id = acc.get("id")
        accounts.append((acc_id, currency, balance))
        
account_df = pd.DataFrame(accounts, columns = ["AccountId", "Currency", "Amount"])
account_df = account_df[account_df.Currency != 'USDT'] # USDT is stablecoin and not an invetment 
account_df

,AccountId,Currency,Amount
0,9119d64c-4c4b-408a-9d78-4ab56686323d,AVAX,19.538485
1,5caae125-471e-4f9f-81c3-5a518e84bcf8,AXS,2.499000
2,2dcbcf3f-92fb-450b-a401-4b253e5c764d,ETH,0.785130
3,acc8cb52-9357-48f5-84c5-f1f9ef47025c,EUR,40.040866
4,d0cbf0c0-6a58-48e9-9c25-6313f66698a0,GALA,1509.000000
5,81705a3d-4ee3-4b72-98ed-bb51d1c6d944,LINK,48.489853
6,2945fd72-4a32-4474-b38e-a3a60cea06c4,MANA,107.280000
7,d6671feb-b1f6-482f-ad4c-6eabfc1ec161,MATIC,505.000000
8,37983bdd-077b-40c7-8ed6-2e78cb06134a,SOL,6.684000


In [6]:
# Retrieving orders and deposits data for each crypto asset
orders = []
deposits = []
for num, acc_id in enumerate(account_df["AccountId"]): # Looping over each crypto asset
    acc_history = client.get_account_history(acc_id)
    for hist in acc_history:
        if  account_df["Currency"][num] == "USD" or account_df["Currency"][num] == "EUR": # USD and EUR transactions go under deposits data
            dicti = dict(hist) # Dictionary that hold data about transactions
            coin = account_df["Currency"][num]
            date = dicti.get("created_at")[0:10]
            amount = round(float(dicti.get("amount")), 2)
            trans_type = dicti.get("type")
            deposits.append((date, coin, trans_type, amount))
            
        else:
            dicti = dict(hist) # Retrieving each crypto asset trading data
            coin = account_df["Currency"][num]
            transfer_type = dicti.get('type') # Saving transfer type in order to see if I bought, send or received the coin
            date = dicti.get("created_at")[0:10]
            amount = float(dicti.get("amount"))
            order_id = dicti.get("details").get("order_id")
            orders.append((date, coin, order_id, transfer_type, amount))


            
orders_df = pd.DataFrame(orders, columns = ["Order Date", "Crypto Currency", "OrderId", 'Transfer Type', "Order Amount"]).set_index("Order Date")
deposits_df = pd.DataFrame(deposits, columns = ["Deposit Date", "Currency", "Transaction Type", "Deposit Amount"]).set_index("Deposit Date")

# Changing match, transfer to buy, send, receive - more intuitive 
orders_df.loc[orders_df['Transfer Type'] == 'match', 'Transfer Type'] = 'Buy'
orders_df.loc[(orders_df['Transfer Type'] == 'transfer') & (orders_df['Order Amount'] < 0), 'Transfer Type'] = 'Send'
orders_df.loc[(orders_df['Transfer Type'] == 'transfer') & (orders_df['Order Amount'] > 0), 'Transfer Type'] = 'Receive'

In [7]:
orders_df.sort_index().head()

,Crypto Currency,OrderId,Transfer Type,Order Amount
Order Date,,,,
2021-08-07,LINK,052c70b4-fdf6-4e59-93cc-2b613d440b4b,Buy,14.240000
2021-09-22,SOL,f1341462-7e72-4846-b19a-7954747dce2e,Buy,0.984000
2021-10-29,ETH,fd5bab87-aae8-469d-9227-13ee90c96b3f,Buy,0.072559
2021-10-29,ETH,None,Send,-0.072559
2021-10-29,ETH,None,Receive,0.063927


In [8]:
deposits_df.head()

,Currency,Transaction Type,Deposit Amount
Deposit Date,,,
2022-05-26,EUR,fee,-0.18
2022-05-26,EUR,match,-45.32
2022-05-26,EUR,fee,-0.22
2022-05-26,EUR,match,-54.86
2022-05-26,EUR,fee,-0.56


### Retrieving data about investment values at transactions and today (Coinbase Pro)

In [9]:
order_list = []

# Functions that eliminates currency from crypto name string
def coin(currency):
    coin = currency.split("-")[0]
    return coin

# Retrieving crypto prices at transactions from Coinbase Pro public API

for num, orderid in enumerate(orders_df.OrderId):
    coin = orders_df["Crypto Currency"][num]
    cur_price = float(public_client.get_product_ticker(product_id = coin + "-EUR").get("price"))
    if orderid != None: 
        order = client.get_order(orderid)
        if order.get("status") == "done":
            product_id = order.get("product_id").split("-")[0]
            action = order.get("side")
            date = order.get("created_at")[0:10]
            amount = float(order.get("filled_size"))
            value = round(float(order.get("executed_value")), 2)
            fees = round(float(order.get("fill_fees")), 2)
            cur_value = cur_price * amount
            order_list.append((date, product_id, action, amount, value, fees, cur_value))
    
    else: # Orderid id none means that crypto was sent to the Coinbase Pro account from another user/exchange
        
        product_id = orders_df["Crypto Currency"][num]
        amount = orders_df["Order Amount"][num]
        if amount > 0:
            action = "receive"
            fees = 0
            
        else:
            action = "send"
            fees = None
        
        
        # Retrieving historic prices for coins that where sent to Coinbase Pro account. Getting the close price of the day when coins where received.
        raw_date = dateutil.parser.isoparse(orders_df.index[num]) 
        date = raw_date.strftime('%Y-%m-%d %H:%M:%S')
        date_dict = {"date": date}
        
        price_req = requests.get("https://api.coinbase.com/v2/prices/" + coin + "-EUR/spot/",params = date_dict)
        res = json.loads(price_req.text)
        price = float(res["data"]["amount"])
        
        value = amount * price
        date = date[0:10]
        cur_value = cur_price * amount
        
        order_list.append((date, product_id, action, amount, value, fees, cur_value))

portfolio_trans = pd.DataFrame(order_list, columns = ["Trans. Date", "Crypto Currency", "Action", "Coin Amount","Value At Transaction", "Fees", "Current Value"]).set_index("Trans. Date")
portfolio_trans = portfolio_trans.drop_duplicates() # With first MANA order, coinbase 'acidentally' put two oders with same id. The coin amount was summed which is false
portfolio_trans.loc[portfolio_trans['Coin Amount'] == 0.28039932, 'Value At Transaction'] = 493.70
portfolio_trans.loc[portfolio_trans['Coin Amount'] == 41.249853, 'Value At Transaction'] = 837.58
portfolio_trans.head() # The resulting dataframe shows the investment value at transaction and today

,Crypto Currency,Action,Coin Amount,Value At Transaction,Fees,Current Value
Trans. Date,,,,,,
2022-05-26,AVAX,buy,2.00000,45.320000,0.18,47.360000
2022-05-18,AVAX,buy,2.00000,58.000000,0.23,47.360000
2022-05-18,AVAX,receive,0.05081,1.655393,0.00,1.203183
2022-05-13,AVAX,buy,4.00000,127.400000,0.76,94.720000
2022-03-09,AVAX,buy,1.50000,106.590000,0.64,35.520000


In [11]:
# Function that makes sell actions appear as negative 
def crypto_sell(action, column):
    if action == 'sell':
        return -column
    else:
        return column

In [12]:
# Applying the above function
portfolio_trans['Coin Amount'] = portfolio_trans.apply(lambda row: crypto_sell(row['Action'], row['Coin Amount']), axis = 1)
portfolio_trans['Value At Transaction'] = portfolio_trans.apply(lambda row: crypto_sell(row['Action'], row['Value At Transaction']), axis = 1)
portfolio_trans['Current Value'] = portfolio_trans.apply(lambda row: crypto_sell(row['Action'], row['Current Value']), axis = 1)

In [13]:
portfolio_trans.head()

,Crypto Currency,Action,Coin Amount,Value At Transaction,Fees,Current Value
Trans. Date,,,,,,
2022-05-26,AVAX,buy,2.00000,45.320000,0.18,47.360000
2022-05-18,AVAX,buy,2.00000,58.000000,0.23,47.360000
2022-05-18,AVAX,receive,0.05081,1.655393,0.00,1.203183
2022-05-13,AVAX,buy,4.00000,127.400000,0.76,94.720000
2022-03-09,AVAX,buy,1.50000,106.590000,0.64,35.520000


# Kraken

### Getting investment data from Kraken account

In [14]:
# Setting up authentification function
import urllib.parse
import hashlib
import hmac
import base64
import pandas as pd
import time
import time
import os
import requests

def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [15]:
# Function that attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

In [16]:
# Retrieving deposits and orders data

crypto_transactions = []
cash_transactions = [] 
resp = kraken_request('/0/private/Ledgers', {
    "nonce": str(int(1000*time.time())),
}, api_key, api_sec)

response = resp.json()
data = response.get('result').get('ledger')
for order in data:
    id_ = order
    tran = data.get(id_)
    
    refid = tran.get('refid')
    time = tran.get('time')
    typ = tran.get('type')
    asset = tran.get('asset')
    amount = round(float(tran.get('amount')), 2)
    fee = round(float(tran.get('fee')), 2)
    balance = round(float(tran.get('balance')), 2)
    
    # Separating crypto and cash transactions to find price at transaction
    if 'EUR' in asset:
        cash_transactions.append((time, refid, typ, asset, amount, fee, balance)) 
    else:
        crypto_transactions.append((time, refid, typ, asset, amount, fee, balance))
    
crypto_df = pd.DataFrame(crypto_transactions, columns = ['Time', 'Transaction ID', 'Type', 'Asset', 'Amount', 'Fee', 'Balance'])
cash_trans = pd.DataFrame(cash_transactions, columns = ['Time', 'Transaction ID', 'Type', 'Asset', 'Amount', 'Fee', 'Balance'])
cash_trans.Time = cash_trans.Time.apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
cash_trans['Fee'] = cash_trans['Fee']* (-1) # Making fee amount negative

In [17]:
# Cash transactions
cash_trans.head()

,Time,Transaction ID,Type,Asset,Amount,Fee,Balance
0,2022-05-26,TSZ2CZH-K2VTO-6LTIB5,spend,ZEUR,-29.56,-0.44,50.0
1,2022-05-18,TS6RCRP-Q6BWR-JOJR7T,spend,ZEUR,-19.70,-0.30,80.0
2,2022-05-15,TSCZU72-A5FR6-VRJB5Z,spend,ZEUR,-49.26,-0.74,100.0
3,2022-05-14,TS7NJEL-SBLWQ-VTIARU,spend,ZEUR,-49.26,-0.74,150.0
4,2022-05-13,QDFFCVS-VPJI25-MRWCSA,deposit,ZEUR,200.00,-0.00,200.0


In [18]:
# Crypto transactions
crypto_df.head()

,Time,Transaction ID,Type,Asset,Amount,Fee,Balance
0,1.653585e+09,TSZ2CZH-K2VTO-6LTIB5,receive,YGG,52.78,0.0,52.78
1,1.652930e+09,TS6RCRP-Q6BWR-JOJR7T,receive,SAND,17.29,0.0,549.06
2,1.652634e+09,TSCZU72-A5FR6-VRJB5Z,receive,SAND,37.42,0.0,531.77
3,1.652551e+09,TS7NJEL-SBLWQ-VTIARU,receive,SAND,39.68,0.0,494.34
4,1.638456e+09,TSXDRML-CZ5NR-O47UPE,receive,SAND,71.22,0.0,454.66


In [19]:
# Adding transaction amount in EUR to crypto transactions table and calculating crypto price at transaction
transactions = crypto_df.merge(cash_trans[['Transaction ID', 'Amount']], how = 'left', on = 'Transaction ID').rename(columns = {'Amount_x':'Coin Amount', 'Amount_y':'Cash', 'Balance': 'Coin Balance'})
transactions['Cash'] = transactions['Cash'] * (-1)
transactions['Price'] = transactions['Cash']/transactions['Coin Amount']
transactions = transactions[['Time', 'Transaction ID', 'Asset', 'Type', 'Coin Amount', 'Price',  'Cash', 'Coin Balance']]
transactions.Time = transactions.Time.apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
transactions

,Time,Transaction ID,Asset,Type,Coin Amount,Price,Cash,Coin Balance
0,2022-05-26,TSZ2CZH-K2VTO-6LTIB5,YGG,receive,52.78,0.560061,29.56,52.78
1,2022-05-18,TS6RCRP-Q6BWR-JOJR7T,SAND,receive,17.29,1.139387,19.70,549.06
2,2022-05-15,TSCZU72-A5FR6-VRJB5Z,SAND,receive,37.42,1.316408,49.26,531.77
3,2022-05-14,TS7NJEL-SBLWQ-VTIARU,SAND,receive,39.68,1.241431,49.26,494.34
4,2021-12-02,TSXDRML-CZ5NR-O47UPE,SAND,receive,71.22,5.533277,394.08,454.66
5,2021-11-11,TSBJVZB-BSVVA-7IH5VZ,SAND,receive,45.12,2.183511,98.52,383.44
6,2021-10-30,TS44N2H-VDTL3-V2PA4U,SAND,spend,-83.25,1.219219,-101.50,338.32
7,2021-09-09,TSDEAZE-UL2C6-UGUMVH,SAND,receive,421.58,0.701077,295.56,421.58


### Reformating Kraken's cash transactions to the Coinbase format
Remormating Kraken API's output was necessary because Kraken's API differs from Coinbase's API

In [20]:
# Reformating Kraken fee dataframe
fee_df = cash_trans[['Time', 'Asset', 'Fee']]
fee_df['Transaction Type'] = 'fee'
fee_df['Asset'] = fee_df['Asset'].map({'ZEUR' : 'EUR'})
fee_df = fee_df.rename(columns = {'Fee' : 'Deposit Amount', 'Asset' : 'Currency', 'Time' : 'Deposit Date'})
fee_df.head()

,Deposit Date,Currency,Deposit Amount,Transaction Type
0,2022-05-26,EUR,-0.44,fee
1,2022-05-18,EUR,-0.30,fee
2,2022-05-15,EUR,-0.74,fee
3,2022-05-14,EUR,-0.74,fee
4,2022-05-13,EUR,-0.00,fee


In [21]:
# Reformating Kraken cash transaction dataframe
tran_df = cash_trans[['Time', 'Type', 'Amount']]
tran_df['Currency'] = 'EUR'
tran_df = tran_df.rename(columns = {'Amount' : 'Deposit Amount', 'Type': 'Transaction Type', 'Time' : 'Deposit Date'})
tran_df.head()

,Deposit Date,Transaction Type,Deposit Amount,Currency
0,2022-05-26,spend,-29.56,EUR
1,2022-05-18,spend,-19.70,EUR
2,2022-05-15,spend,-49.26,EUR
3,2022-05-14,spend,-49.26,EUR
4,2022-05-13,deposit,200.00,EUR


In [22]:
# Merging cash and fee transactions which results in cash transactions table that is identical to Coinbase's format
cash_df = pd.concat([tran_df, fee_df]).set_index('Deposit Date')
cash_df['Transaction Type'].replace({'spend': 'match', 'deposit': 'transfer'}, inplace = True)
cash_df.sort_index().head()

,Transaction Type,Deposit Amount,Currency
Deposit Date,,,
2021-09-09,transfer,300.00,EUR
2021-09-09,fee,-4.44,EUR
2021-09-09,match,-295.56,EUR
2021-09-09,fee,-0.00,EUR
2021-10-30,receive,101.50,EUR


In [23]:
# Merging deposits data from Coinbase and Kraken together
deposits = pd.concat([deposits_df, cash_df]).sort_index()
deposits.head()

,Currency,Transaction Type,Deposit Amount
Deposit Date,,,
2021-08-03,EUR,transfer,4.00
2021-08-05,EUR,transfer,300.00
2021-08-07,EUR,match,-303.63
2021-08-07,EUR,fee,-0.12
2021-09-09,EUR,match,-295.56


### Reformating Kraken's crypto transactions to the Coinbase's format

In [24]:
# Function that calculates the current value of each investment 
def cur_value(df):
    resp = requests.get('https://api.kraken.com/0/public/Ticker?pair=' + df['Crypto Currency'] + 'EUR').json()
    cur_price = resp['result'][df['Crypto Currency'] + 'EUR']['a'][0]
    cur_value = float(cur_price) * df['Coin Amount']
    return cur_value

In [25]:
# Renaming Kraken's dataframe columns in a way that matches Coinbase datafame and applying the previous function
transactions = transactions[['Time', 'Asset', 'Type', 'Coin Amount', 'Cash']]
transactions = transactions.rename(columns = {'Asset': 'Crypto Currency', 'Time': 'Trans. Date', 'Type': 'Action', 'Cash': 'Value At Transaction'})
transactions['Fees'] = abs(transactions['Value At Transaction'] * 0.015) 
transactions['Action'].replace({'spend': 'sell', 'deposit': 'buy'}, inplace = True)
transactions['Current Value'] = transactions.apply(cur_value, axis = 1)
transactions.set_index('Trans. Date', inplace = True)
transactions.head()

,Crypto Currency,Action,Coin Amount,Value At Transaction,Fees,Current Value
Trans. Date,,,,,,
2022-05-26,YGG,receive,52.78,29.56,0.4434,30.031820
2022-05-18,SAND,receive,17.29,19.70,0.2955,20.786038
2022-05-15,SAND,receive,37.42,49.26,0.7389,44.978840
2022-05-14,SAND,receive,39.68,49.26,0.7389,47.703296
2021-12-02,SAND,receive,71.22,394.08,5.9112,85.606440


In [26]:
# Blending Ccoinbase Pro and Kraken transactions
crypto_transactions = pd.concat([transactions, portfolio_trans]).sort_index()
crypto_transactions.head()

,Crypto Currency,Action,Coin Amount,Value At Transaction,Fees,Current Value
Trans. Date,,,,,,
2021-08-07,LINK,buy,14.240000,303.630000,0.1200,89.000000
2021-09-09,SAND,receive,421.580000,295.560000,4.4334,506.739160
2021-09-22,SOL,buy,0.984000,109.450000,0.5500,40.629360
2021-10-29,ETH,send,-0.113675,-417.324797,NaN,-189.921369
2021-10-29,ETH,buy,0.067435,251.990000,1.2600,112.666770


In [27]:
# Downloading crypto transactions data
crypto_transactions.to_csv("cryptoData/cryptoTransactions.csv", index = True)

In [28]:
# Downloading cash transactions data
deposits.to_csv("cryptoData/currencyTransactions.csv", index = True)

# Feature Engineering on Combined Data

### Calculating weighted average prices for each crypto 

In [29]:
# Calculating crypto price and transaction
port_copy = crypto_transactions.copy()
port_copy['Price At Transaction'] = port_copy['Value At Transaction']/port_copy['Coin Amount'] # Price at transaction
port_copy = port_copy[['Crypto Currency', 'Coin Amount', 'Price At Transaction', 'Value At Transaction']]
port_copy = port_copy.loc[port_copy['Coin Amount'] > 0] # Filtering out send and receive transactions
port_copy = port_copy.drop_duplicates()
port_copy.head()

,Crypto Currency,Coin Amount,Price At Transaction,Value At Transaction
Trans. Date,,,,
2021-08-07,LINK,14.240000,21.322331,303.630000
2021-09-09,SAND,421.580000,0.701077,295.560000
2021-09-22,SOL,0.984000,111.229675,109.450000
2021-10-29,ETH,0.067435,3736.769716,251.990000
2021-10-29,ETH,0.063927,3671.210000,234.689442


In [30]:
port_copy.to_csv("cryptoData/Prices.csv", index = True)

In [31]:
# Function that calculates weighted average prices
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [32]:
# Applyinh the function to get weighted average prices
avg_prices = pd.DataFrame(port_copy.groupby('Crypto Currency').apply(w_avg, 'Price At Transaction', 'Coin Amount')).reset_index()
avg_prices.rename(columns = {0: 'Weighted Avg Price'}, inplace = True)
avg_prices

,Crypto Currency,Weighted Avg Price
0,AVAX,70.566122
1,AXS,19.291717
2,ETH,2679.170738
3,GALA,0.368767
4,LINK,19.073337
5,MANA,3.112135
6,MATIC,1.387545
7,SAND,1.433442
8,SOL,115.909635
9,YGG,0.560061


### Retrieving current prices to find current investment values

In [33]:
# Finding total coin amount of each asset in the portfolio
accounts = crypto_transactions.groupby(by = 'Crypto Currency').sum().reset_index()[['Crypto Currency','Coin Amount']].rename(columns = {'Crypto Currency': 'Currency', 'Coin Amount': 'Amount'})

# Using Coinbase Pro public API to get current prices of assets that are in the portfolio
price_list = []
time = public_client.get_product_ticker(accounts.Currency[0] + "-EUR").get("time").split("T")[0]
for tick in accounts.Currency:
    if tick != "EUR" and tick != "USD" and tick != 'SAND' and tick != 'YGG':
        ticker = public_client.get_product_ticker(product_id=tick + "-EUR")
        coin = tick
        time = time
        price = float(ticker.get("price"))
        price_list.append((time, coin, price))
        
    elif tick == 'SAND' or tick == 'YGG': # Coinbase Pro does not have SAND and YGG data so I have to make a Kraken request
        ticker = requests.get('https://api.kraken.com/0/public/Ticker?pair=' +  'SANDEUR').json()
        coin = tick
        time = time
        price = float(ticker['result']['SANDEUR']['a'][0])
        price_list.append((time, coin, price))
        

price_df = pd.DataFrame(price_list, columns = ["Current Date", "Currency", "Current Price"])
portfolio = price_df.merge(accounts[["Currency", "Amount"]], how = "left", on = "Currency").rename(columns = {"Amount": "Coin Amount"})
portfolio["Value Today"] = round(portfolio["Current Price"] * portfolio["Coin Amount"], 2)
portfolio.head()

,Current Date,Currency,Current Price,Coin Amount,Value Today
0,2022-05-29,AVAX,23.6000,19.538485,461.11
1,2022-05-29,AXS,16.7700,2.499000,41.91
2,2022-05-29,ETH,1668.3600,0.785130,1309.88
3,2022-05-29,GALA,0.0703,1509.000000,106.08
4,2022-05-29,LINK,6.2400,48.489853,302.58


In [34]:
# Creating a table that shows how much has been invested into each crypto asset
tran_value = pd.DataFrame(crypto_transactions.groupby('Crypto Currency').sum()['Value At Transaction']).reset_index()
tran_value.head()

,Crypto Currency,Value At Transaction
0,AVAX,1243.467827
1,AXS,48.210000
2,ETH,1918.747007
3,GALA,556.470000
4,LINK,879.690000


In [35]:
# Creating a portfolio table 
portfolio = pd.merge(portfolio, avg_prices, how = "left", left_on = ["Currency"], right_on = ["Crypto Currency"]) # adding weighted average prices to portfolio table
portfolio = pd.merge(portfolio, tran_value, how = "left", left_on = ["Currency"], right_on = ["Crypto Currency"]) # adding transaction values to portfolio table
portfolio.set_index("Current Date", inplace = True)
portfolio.drop(columns = ["Crypto Currency_x", 'Crypto Currency_y'], inplace = True)
portfolio.head()

,Currency,Current Price,Coin Amount,Value Today,Weighted Avg Price,Value At Transaction
Current Date,,,,,,
2022-05-29,AVAX,23.6000,19.538485,461.11,70.566122,1243.467827
2022-05-29,AXS,16.7700,2.499000,41.91,19.291717,48.210000
2022-05-29,ETH,1668.3600,0.785130,1309.88,2679.170738,1918.747007
2022-05-29,GALA,0.0703,1509.000000,106.08,0.368767,556.470000
2022-05-29,LINK,6.2400,48.489853,302.58,19.073337,879.690000


### Retrieving crypto assets trading volume and market cap data 

In [36]:
# Using CoinMarketCap API to get trading volume and market cap data
key = "547412f5-c128-4f00-9ab8-15e7b57456ca"
def market_cap_vol(coin):

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    parameters = {
        "symbol": coin.upper(),
        "convert":"EUR"
    }
    
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': key,
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)

    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    volume = float(round(data["data"][coin.upper()]["quote"]["EUR"]["volume_24h"],0))
    market_cap = float(round(data["data"][coin.upper()]["quote"]["EUR"]["market_cap"],0))
    
    return [market_cap, volume]
  

In [37]:
# Function that retrieves volume and market cap data for each crypto asset
def cap_vol(row):
    cap = market_cap_vol(row["Currency"])
    return cap

In [38]:
# Applying the function
portfolio["cap_vol"] = portfolio.apply(cap_vol, axis = 1)
split_df = pd.DataFrame(portfolio["cap_vol"].to_list(), columns = ["Market Cap", "24h Volume"])
portfolio["Market Cap"] = list(split_df["Market Cap"].values)
portfolio["24h Volume"] = list(split_df["24h Volume"].values)
portfolio.drop(columns = ["cap_vol"], inplace = True)
portfolio.head()

,Currency,Current Price,Coin Amount,Value Today,Weighted Avg Price,Value At Transaction,Market Cap,24h Volume
Current Date,,,,,,,,
2022-05-29,AVAX,23.6000,19.538485,461.11,70.566122,1243.467827,6.417369e+09,6.423741e+08
2022-05-29,AXS,16.7700,2.499000,41.91,19.291717,48.210000,1.024673e+09,1.077094e+08
2022-05-29,ETH,1668.3600,0.785130,1309.88,2679.170738,1918.747007,2.021029e+11,9.287570e+09
2022-05-29,GALA,0.0703,1509.000000,106.08,0.368767,556.470000,4.917017e+08,1.474092e+08
2022-05-29,LINK,6.2400,48.489853,302.58,19.073337,879.690000,2.910619e+09,2.224940e+08


In [39]:
portfolio.to_csv("cryptoData/portfolio.csv", index = True)

### Retrieving historic price data to see the performance of investments

In [40]:
# Creating a table that helps to calculate portfolio value over time
portfolio_df = crypto_transactions.reset_index()
portf_helper = portfolio_df.groupby(by = ["Trans. Date", "Crypto Currency"]).sum()
portf_helper.reset_index(inplace = True)
portf_helper.set_index("Trans. Date", inplace = True)
portf_helper.head()

,Crypto Currency,Coin Amount,Value At Transaction,Fees,Current Value
Trans. Date,,,,,
2021-08-07,LINK,14.240000,303.630000,0.1200,89.000000
2021-09-09,SAND,421.580000,295.560000,4.4334,506.739160
2021-09-22,SOL,0.984000,109.450000,0.5500,40.629360
2021-10-29,ETH,0.017687,71.637007,2.6000,29.550796
2021-10-30,SAND,-83.250000,-101.500000,1.5225,-100.066500


In [41]:
#exp. ticker = BTC, LINK, AVAX
#exp. currency = USD, EUR

# Creating a function that retrieves daily data (high, low, open, close) for each asset in the portfolio
def get_portfolio(ticker, currency, n_days):
    
    url = "https://min-api.cryptocompare.com/data/v2/histoday?fsym=" + ticker + "&tsym=" + currency + "&limit=" + str(n_days)
    response = requests.get(url)   
    data = response.json()['Data']['Data'] 
    df = pd.DataFrame.from_dict(data)
 
    # time is stored as an epoch, we need normal dates
    df["Date"] = pd.to_datetime(df['time'], unit='s')
    df = df[["Date", 'high', 'low', 'open','close', 'volumeto']]
    df.set_index("Date", inplace = True)

    def coin(currency):
        coin = currency.split("-")[0]
        return coin
    

    coin_df = portf_helper[["Crypto Currency", "Coin Amount", 'Value At Transaction']]
    coin_df["Crypto Currency"] = coin_df["Crypto Currency"].apply(coin)
    coin_df = coin_df.loc[coin_df["Crypto Currency"] == ticker]
    coin_df.index = pd.to_datetime(coin_df.index)
    joined_df = df.join(coin_df, how = "outer")
    
    joined_df["Coin Amount"] = joined_df["Coin Amount"].fillna(0)
    joined_df["Coin Amount"] = joined_df["Coin Amount"].cumsum(axis = 0)
    
    joined_df["Value At Transaction"] = joined_df["Value At Transaction"].fillna(0)
    joined_df["Value At Transaction"] = joined_df["Value At Transaction"].cumsum(axis = 0)
    
    joined_df["Crypto Currency"] = joined_df["Crypto Currency"].fillna("-")
    joined_df["Investment Value"] = round(joined_df["close"] * joined_df["Coin Amount"], 2)
    
    joined_df['Net P&L'] = joined_df["Investment Value"] - joined_df["Value At Transaction"]
    
    joined_df["Label"] = ticker
    
    joined_df = joined_df.rename(columns = {"open": "Open", "close": "Close", "high": "High", "low": "Low", 'volumeto': "Volume"})
    joined_df.reset_index(inplace = True)
    joined_df.rename(columns = {"index": "Hist. Date"}, inplace = True)
    
    
 
    return joined_df

In [42]:
# Applying the funciton
df_list = []
for coin in portfolio['Currency']:
    df = get_portfolio(coin, "EUR", 300)
    df_list.append(df)
coin_prices = pd.concat(df_list)  

In [43]:
coin_prices.tail()

,Hist. Date,High,Low,Open,Close,Volume,Crypto Currency,Coin Amount,Value At Transaction,Investment Value,Net P&L,Label
296,2022-05-25,0.6572,0.6111,0.6373,0.6382,166870.53,-,0.00,0.00,0.00,0.00,YGG
297,2022-05-26,0.5954,0.5320,0.6431,0.5370,3079.59,YGG,52.78,29.56,28.34,-1.22,YGG
298,2022-05-27,0.5534,0.5057,0.5339,0.5401,164498.46,-,52.78,29.56,28.51,-1.05,YGG
299,2022-05-28,0.6547,0.5612,0.5399,0.5779,7330.49,-,52.78,29.56,30.50,0.94,YGG
300,2022-05-29,0.5920,0.5360,0.5779,0.5663,13131.58,-,52.78,29.56,29.89,0.33,YGG


In [44]:
coin_prices.to_csv("cryptoData/coinData.csv", index = True)

In [45]:
coin_prices.rename(columns = {'Hist. Date' : 'Date'}, inplace = True)

### Scraping crypto market fear data 

In [46]:
# Making a request to cryptocurrencytracker website
fear = 'https://cryptocurrencytracker.info/en/fear-and-greed-index'
page = requests.get(fear, headers={ "user-agent": 'Mozilla/5.0'})
soup = BeautifulSoup(page.content, "html.parser")

fear_level = soup.find_all('div', class_ = 'left')
fear_nr = soup.find_all('div', class_ = 'right')
time  = ['Now', 'Yesterday', 'Last Week', 'Last Month']

fear_list = [] # Retrieving ('Now', 'Yesterday', 'Last Week', 'Last Month') fear data

for i in range(len(fear_level)):
    fear_list.append((time[i], fear_level[i].text, int(fear_nr[i].text)))

fear_df = pd.DataFrame(fear_list, columns = ['Time', 'Fear Level', 'Score'])
fear_df.set_index('Time', inplace = True)
fear_df

,Fear Level,Score
Time,,
Now,Extreme Fear,14
Yesterday,Extreme Fear,13
Last Week,Extreme Fear,14
Last Month,Extreme Fear,23


In [55]:
fear_df.to_csv("cryptoData/fearData.csv", index = True)

### Retrieving overall crypto market data

In [115]:
# Saving the data to a dictionary
root_dict = {}

gecko = 'https://www.coingecko.com/'
page = requests.get(gecko, headers={"user-agent": 'Mozilla/5.0'})
soup = BeautifulSoup(page.content, "html.parser")

# Retrieving crypto market data
market_info = soup.find_all('dt', class_ = "tw-inline-flex tw-items-center tw-text-xl tw-font-medium tw-text-gray-900 tw-truncate dark:tw-text-white dark:tw-text-opacity-87")
metric_list = []
for i in info:
    if len(i.find_all('span')) !=0:
        for html in i.find_all('span'):
            val = html.text.replace('\n','').replace('%','')
            metric_list.append(val)

    else:
        val = i.text.replace('\n', '').replace('%','')
        metric_list.append(val) 
        
metric_list

['$1,273,439,098,167', '0.6', '$49,848,768,046', '43.76', '13442']

In [116]:
root_dict['Market'] = metric_list

In [117]:
with open('cryptoData/cryptoJSON.json', 'w') as json_file:
    json.dump(root_dict, json_file)
    
root_dict

{'Market': ['$1,273,439,098,167', '0.6', '$49,848,768,046', '43.76', '13442']}

In [ ]:
# Retrieving Bitcoin and Ethereum data (two biggest crypto assets)
coins = soup.find_all('tr')[1:3]

coin_list = []
for coin in coins:
    one = []
    c = coin.find_all('span')
    for i in c:
        try:
            one.append(float(i.text.strip().replace('%', '')))
        except:
            one.append(i.text.strip().replace('%', ''))
        
    coin_list.append(one)
coin_list

In [50]:
# Saving the data to a dataframe
btc_eth = pd.DataFrame(list(zip(coin_list[0], coin_list[1]))).T.round(1)
btc_eth.columns = ['Coin', 'Price', '1h', '24h', '7d', '24h Volume', 'Mkt Cap']
btc_eth

,Coin,Price,1h,24h,7d,24h Volume,Mkt Cap
0,BTC,"$29,176.34",0.1,0.5,-1.1,"$13,317,601,341","$555,313,182,997"
1,ETH,"$1,795.43",0.3,0.2,-9.2,"$9,157,902,276","$216,465,562,093"


In [51]:
btc_eth.to_csv('cryptoData/coinTable.csv')

### Scraping traditional market indexes data to get an overview of traditional markets

In [52]:
# Scraping index data from Yahoo Finance
yahoo = 'https://finance.yahoo.com/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAANCpEZowy6FIk2H6-yaZ_ThU5V4RplEogh9yLY_sGx6G9QEcAfjBN1w-lakmH8vLwFY2O7M9QaxbZYHT_5GBAiFkFcH-snBwzzdm-RiXSCg_Nm2JxQovImd79EjAchv_c0xtvwZEKVGmfl1WIGo2YtX8Rlcirg0uCY0n1vsvybfC'
page = requests.get(yahoo, headers={ "user-agent": 'Mozilla/5.0'})
soup = BeautifulSoup(page.content, "html.parser")
index = soup.find('ul', class_ = 'Carousel-Slider Pos(r) Whs(nw)')

# Indexes' values
idx_price = soup.find_all('fin-streamer', class_ = 'Fz(s) Mt(4px) Mb(0px) Fw(b) D(ib)')
price = []
for i in idx_price:
    price.append(float(i.text.replace(',', '')))

# Indexes' names
idx_name = soup.find_all('a', class_ = 'Fz(s) Ell Fw(600) C($linkColor)')
names = []
for i in idx_name:
    names.append(str(i.text))

# Indexes' point and percent changes
points = soup.find('ul', class_ = 'Carousel-Slider Pos(r) Whs(nw)').find_all('span')
point = []
change = []
for i, val in enumerate(points):
    if i%2 != 0:
        change.append(float(val.text[1:-2]))
    else:
        point.append(float(val.text))



In [53]:
# Creating an index dataframe
dicti= {'Name': idx_name, 'Value': price, 'Point Change': point, '% Change': change}
index_df = pd.DataFrame(dicti)
index_df['Name'] = index_df['Name'].apply(lambda x: x.text)
index_df.set_index('Name', inplace = True)
index_df

,Value,Point Change,% Change
Name,,,
S&P 500,4158.24,100.40,2.47
Dow 30,33212.96,575.76,1.76
Nasdaq,12131.13,390.43,3.33
Russell 2000,1887.90,49.66,2.70


In [54]:
index_df.to_csv("cryptoData/indexData.csv", index = True)